In [1]:
import os
import sys
import glob
import math
import time
import json
import importlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
projFol = os.path.dirname(os.getcwd())
modelsFol = os.path.join(projFol,"models")
dataFol = os.path.join(projFol,"data")

print("Current working directory:\n",os.getcwd())
print("\nProj folder:\n",projFol,"\nModels folder:\n",modelsFol,"\nData folder;\n",dataFol)

# sys.path.append(modelsFol)
# print("\nSystem Path folders:")
# sys.path

# importlib.reload(clean)

Current working directory:
 C:\Users\Craig D\Code

Proj folder:
 C:\Users\Craig D 
Models folder:
 C:\Users\Craig D\models 
Data folder;
 C:\Users\Craig D\data
